Import packages

In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
# import tensorflow as tf
from sklearn.preprocessing import LabelEncoder
import torch
import torch.nn as nn
import torch.optim as optim
import matplotlib.pyplot as plt
from sklearn.inspection import permutation_importance
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score

from sklearn.model_selection import StratifiedKFold

Load data

In [139]:
read_train_data = pd.read_csv('../champ_result/GSE243529_train/all_beta_normalized_train.csv')

In [165]:
read_train_data.rename(columns={'Unnamed: 0':'id'}, inplace=True)
train_data = read_train_data.iloc[:,::2]
tumor_num = 128
normal_num = 134
# read_test_data = pd.read_csv('../result/all_beta_normalized_test_0.4.csv')
# X = pd.concat([read_train_data.iloc[:,1::2], read_test_data.iloc[:,1::2]], axis=1, ignore_index=True)

In [166]:
train_data

,id,2,4,6,8,10,12,14,16,18,...,506,508,510,512,514,516,518,520,522,524
0,cg07881041,0.891637,0.932066,0.940202,0.945641,0.940250,0.932014,0.926817,0.947249,0.928761,...,0.937577,0.936447,0.944013,0.952812,0.934124,0.927144,0.954413,0.960868,0.943230,0.946855
1,cg03513874,0.942312,0.935187,0.964009,0.966716,0.945237,0.943195,0.941343,0.939910,0.957141,...,0.958525,0.946503,0.937960,0.951564,0.952554,0.949742,0.942481,0.964317,0.945584,0.962746
2,cg05451842,0.029975,0.022880,0.017531,0.026281,0.034589,0.030383,0.044970,0.033864,0.044376,...,0.040533,0.023186,0.036200,0.035194,0.021445,0.030706,0.025217,0.019059,0.031791,0.017954
3,cg14797042,0.983277,0.989621,0.987711,0.960712,0.966604,0.966022,0.973924,0.974524,0.963827,...,0.976318,0.972781,0.983394,0.959674,0.963826,0.973674,0.971978,0.968671,0.966904,0.972501
4,cg09838562,0.009447,0.009020,0.007164,0.007786,0.024895,0.032956,0.021911,0.022853,0.004058,...,0.013668,0.018324,0.017871,0.021473,0.011137,0.009642,0.012426,0.020793,0.014402,0.018484
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
736420,cg19812938,0.867959,0.903661,0.890041,0.896655,0.859622,0.875650,0.868883,0.890319,0.879433,...,0.850947,0.882090,0.888067,0.894050,0.874520,0.878742,0.899942,0.897113,0.885093,0.878564
736421,cg06272054,0.008151,0.011305,0.011354,0.009927,0.008303,0.017703,0.021295,0.021937,0.017683,...,0.016715,0.012312,0.015625,0.015002,0.010521,0.016021,0.012377,0.011960,0.006381,0.014298
736422,cg07255356,0.017649,0.016949,0.022004,0.019642,0.027163,0.032100,0.047964,0.032659,0.030648,...,0.028312,0.028406,0.031591,0.020285,0.020617,0.020522,0.013245,0.021643,0.024515,0.029187
736423,cg24220897,0.932095,0.921312,0.946663,0.965407,0.949585,0.953402,0.920959,0.944034,0.931178,...,0.922066,0.935919,0.932090,0.836576,0.915516,0.923381,0.861654,0.904196,0.929799,0.953917


In [150]:
info_data = pd.read_csv('../result/result_basic_dbeta_GSE243529.csv')
info_data.columns = ['id', 'dbeta', 'gene']

In [151]:
info_data = info_data[abs(info_data['dbeta']) > 0.03]
info_data

,id,dbeta,gene
50,cg10523679,-0.030600,ACADM
70,cg13642872,0.030652,ACSBG1
95,cg07568040,0.033123,ACVR1C
105,cg04892170,-0.040451,ADAM12
135,cg20395881,0.033628,ADARB2
...,...,...,...
7635,cg01174743,-0.032216,ZDHHC14
7642,cg00602811,0.039105,ZEB2
7676,cg14037413,-0.056093,ZNF143
7683,cg26649251,-0.042365,ZNF224


In [152]:
ward_labels = pd.read_csv('../result/ward_labels.csv')
ward_labels.columns = ['gene', 'cluster']

In [153]:
info_data = pd.merge(info_data, ward_labels, on='gene', how='inner')

In [154]:
info_data

,id,dbeta,gene,cluster
0,cg10523679,-0.030600,ACADM,3
1,cg13642872,0.030652,ACSBG1,3
2,cg07568040,0.033123,ACVR1C,3
3,cg04892170,-0.040451,ADAM12,2
4,cg20395881,0.033628,ADARB2,1
...,...,...,...,...
294,cg01174743,-0.032216,ZDHHC14,3
295,cg00602811,0.039105,ZEB2,1
296,cg14037413,-0.056093,ZNF143,1
297,cg26649251,-0.042365,ZNF224,1


In [163]:
train_data

,id,2,4,6,8,10,12,14,16,18,...,506,508,510,512,514,516,518,520,522,524
1677,cg20708990,0.412746,0.644426,0.609446,0.617260,0.767255,0.633936,0.587169,0.644283,0.595754,...,0.722999,0.687195,0.669669,0.645648,0.671220,0.689254,0.655640,0.657192,0.742449,0.640686
5181,cg23487721,0.395120,0.678729,0.557116,0.743407,0.710758,0.599495,0.684064,0.778117,0.697982,...,0.808271,0.760352,0.706295,0.711597,0.722357,0.705325,0.746364,0.662604,0.736111,0.712095
7009,cg03465320,0.314291,0.443292,0.389277,0.532550,0.629425,0.365145,0.637896,0.531189,0.583710,...,0.595505,0.676296,0.701537,0.603420,0.425575,0.614176,0.727226,0.507113,0.692644,0.630584
8120,cg11855664,0.366450,0.493737,0.431657,0.499973,0.489960,0.514827,0.561834,0.526685,0.434066,...,0.534519,0.510408,0.507501,0.478970,0.496053,0.509022,0.497867,0.383478,0.576321,0.502746
9420,cg23378941,0.458256,0.466731,0.340482,0.352527,0.479365,0.530109,0.534439,0.477975,0.463129,...,0.479220,0.554817,0.418580,0.491556,0.457081,0.420912,0.416581,0.407134,0.411505,0.434761
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
724591,cg06688763,0.772047,0.844641,0.719520,0.803821,0.648736,0.637732,0.663257,0.515662,0.522579,...,0.360198,0.417587,0.688262,0.390023,0.447691,0.554579,0.501756,0.553143,0.622071,0.459663
729347,cg15100599,0.281005,0.218475,0.175893,0.173109,0.103639,0.201652,0.191372,0.201248,0.252543,...,0.099030,0.234776,0.244935,0.159055,0.091320,0.188719,0.260390,0.224498,0.200165,0.252226
729605,cg26649251,0.042544,0.117241,0.194948,0.072777,0.114303,0.184105,0.183858,0.218480,0.189018,...,0.053213,0.197533,0.214305,0.133006,0.055717,0.161621,0.222377,0.181510,0.150973,0.248852
730407,cg03987199,0.901540,0.896326,0.958006,0.946826,0.931684,0.770218,0.941319,0.912341,0.893889,...,0.857657,0.843952,0.920473,0.923043,0.747709,0.831397,0.950481,0.792176,0.858453,0.852413


rfecv select optimal features for each model

In [168]:
from sklearn.feature_selection import RFECV
from sklearn.model_selection import StratifiedKFold
from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier
from sklearn.metrics import make_scorer

train_data = train_data[train_data['id'].isin(info_data['id'])]

models = {
    # "SVM": SVC(kernel="linear"),
    # "Logistic Regression": LogisticRegression(),
    # "Decision Tree": DecisionTreeClassifier(),
    # "Random Forest": RandomForestClassifier(),
    "XGBoost": XGBClassifier()
}

# row are clusters, columns are models
optimal_cluster_dataframe = pd.DataFrame()
feature_dict = {}

for cluster_idx in range(1, 4):
    model_optimal_feature = {}
    feature_list = []
    for model_name, estimator in models.items():
        print(f"Cluster: {cluster_idx}, Model: {model_name}")
        train_group = train_data[train_data['id'].isin(info_data[info_data['cluster'] == cluster_idx]['id'])]
        x = train_group.iloc[:,1::].reset_index(drop=True)
        x = x.T
        y = np.array([(0 if (i < tumor_num) else 1) for i in range(tumor_num + normal_num)])
        scoror = make_scorer(f1_score, average='macro')
        rfecv = RFECV(estimator=estimator, step=1, cv=StratifiedKFold(5), scoring=scoror)
        rfecv.fit(x, y)
        print("Optimal number of features : %d" % rfecv.n_features_)

        model_optimal_feature[model_name] = rfecv.n_features_
        feature_list.append(rfecv.support_)

    cluster_features_df = pd.DataFrame(model_optimal_feature, index=[f'cluster_{cluster_idx}'])
    feature_dict[f'cluster_{cluster_idx}'] = feature_list
    optimal_cluster_dataframe = pd.concat([optimal_cluster_dataframe, cluster_features_df], axis=0)

Cluster: 1, Model: XGBoost
Optimal number of features : 45
Cluster: 2, Model: XGBoost
Optimal number of features : 94
Cluster: 3, Model: XGBoost
Optimal number of features : 27


In [169]:
optimal_cluster_dataframe

,XGBoost
cluster_1,45
cluster_2,94
cluster_3,27


In [180]:
for cluster_idx in range(1, 4):
    train_group = train_data[train_data['id'].isin(info_data[info_data['cluster'] == cluster_idx]['id'])]
    train_group = train_group[feature_dict[f'cluster_{cluster_idx}'][0]]
    # reset columns
    train_group.columns = range(train_group.shape[1])
    train_group.rename(columns={0:'id'}, inplace=True)
    # train_group.insert(0, 'id', train_data['id'])
    train_group.to_csv(f'../champ_result/GSE243529_train/cluster_{cluster_idx}_train.csv', index=False)

AutoEncoder

In [26]:
X = read_train_data.iloc[:,1::2].T
Y = pd.DataFrame([ (0 if i < tumor_num else 1)  for i in range(tumor_num + normal_num)])

In [27]:
print(X.shape)
print(Y.shape)

(262, 306)
(262, 1)


NN model

In [63]:
class Net(nn.Module):
    def __init__(self, input_size, hidden_size):
        super(Net, self).__init__()
        self.fc1 = nn.Linear(input_size, hidden_size)
        self.middle = nn.Linear(hidden_size, hidden_size)
        self.fc2 = nn.Linear(hidden_size, 1)

        self.sigmoid = nn.Sigmoid()
    def forward(self, x):
        out = self.fc1(x)
        out = self.sigmoid(out)
        out = self.middle(out)
        out = self.sigmoid(out)
        out = self.fc2(out)
        return out

def train(model, train_data, train_labels, test_data, test_labels, num_epochs, learning_rate):
    criterion = nn.MSELoss()
    optimizer = optim.Adam(model.parameters(), lr=learning_rate)
    train_losses = []
    test_losses = []
    for epoch in range(num_epochs):
        model.train()
        optimizer.zero_grad()
        train_outputs = model(train_data)
        train_loss = criterion(train_outputs, train_labels)
        train_loss.backward()
        optimizer.step()
        train_losses.append(train_loss.item())
        model.eval()
        with torch.no_grad():
            test_outputs = model(test_data)
            test_loss = criterion(test_outputs, test_labels)
            test_losses.append(test_loss.item())
        if (epoch + 1) % 10 == 0:
            print(f"Epoch {epoch+1}: Train Loss = {train_loss.item()}, Test Loss = {test_loss.item()}")
    return train_losses, test_losses

def plot_losses(train_losses, test_losses):
    plt.plot(train_losses, label='Train Loss')
    plt.plot(test_losses, label='Test Loss')
    plt.xlabel('Epochs')
    plt.ylabel('Loss')
    plt.legend()
    plt.title('Train and Test Losses')
    plt.show()

In [ ]:
skf = StratifiedKFold(n_splits=10, shuffle=True, random_state=0)
accuracy_list = []
precision_list = []
recall_list = []
f1_list = []
auc_list = []

for X_item, Y_item in skf.split(X, Y):
    X_train, X_test = X.iloc[X_item], X.iloc[Y_item]
    Y_train, Y_test = Y.iloc[X_item], Y.iloc[Y_item]
    
    X_train = torch.tensor(X_train.values, dtype=torch.float32)
    Y_train = torch.tensor(Y_train.values, dtype=torch.float32)
    X_test = torch.tensor(X_test.values, dtype=torch.float32)
    Y_test = torch.tensor(Y_test.values, dtype=torch.float32)

    model = Net(X_train.shape[1], 100)
    
    train_losses, test_losses = train(model, X_train, Y_train, X_test, Y_test, 300, 0.005)

    plot_losses(train_losses, test_losses)

    Y_pred = model(X_test).detach().numpy()
    Y_pred_class = Y_pred.round()  # Assuming binary classification

    accuracy = accuracy_score(Y_test, Y_pred_class)
    precision = precision_score(Y_test, Y_pred_class)
    recall = recall_score(Y_test, Y_pred_class)
    f1 = f1_score(Y_test, Y_pred_class)
    accuracy_list.append(accuracy)
    precision_list.append(precision)
    recall_list.append(recall)
    f1_list.append(f1)
    auc_list.append(roc_auc_score(Y_test, Y_pred_class))
    print(f"Accuracy: {accuracy}")
    print(f"Precision: {precision}")
    print(f"Recall: {recall}")
    print(f"F1 Score: {f1}")
    print(f"auc: {roc_auc_score(Y_test, Y_pred_class)}")

In [65]:
# overall aggregated metrics
print("Overall Aggregated Metrics:")
print(f"Accuracy: {np.mean(accuracy_list)}")
print(f"Precision: {np.mean(precision_list)}")
print(f"Recall: {np.mean(recall_list)}")
print(f"F1 Score: {np.mean(f1_list)}")
print(f"auc: {np.mean(auc_list)}")

Overall Aggregated Metrics:
Accuracy: 0.8283475783475783
Precision: 0.8541971916971918
Recall: 0.8225274725274726
F1 Score: 0.8253054719350603
auc: 0.8292124542124542


When model.eval() is called, the behavior of the model is changed by disabling certain operations like dropout and batch normalization during inference. These operations are often used during training to improve generalization and prevent overfitting, but they are not needed during inference.

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset

# Define the architecture of the autoencoder
input_size = 139
mid_size = 80
encoding_dim = 20

class Autoencoder(nn.Module):
    def __init__(self, input_size, mid_size, encoding_dim):
        super(Autoencoder, self).__init__()
        self.encoder = nn.Sequential(
            nn.Linear(input_size, mid_size),
            nn.ReLU(),
            nn.Linear(mid_size, encoding_dim),
        )
        self.decoder = nn.Sequential(
            nn.Linear(encoding_dim, mid_size),
            nn.Sigmoid(),
            nn.Linear(mid_size, input_size),
            nn.Sigmoid()
        )

    def forward(self, x):
        x = self.encoder(x)
        x = self.decoder(x)
        return x

# Convert your numpy arrays to PyTorch tensors
x_train_tensor = torch.tensor(x_train.values, dtype=torch.float32)
x_test_tensor = torch.tensor(x_test.values, dtype=torch.float32)

# Create DataLoader for training
train_dataset = TensorDataset(x_train_tensor, x_train_tensor)
train_loader = DataLoader(train_dataset, batch_size=256, shuffle=True)

# Initialize the model, loss function, and optimizer
autoencoder = Autoencoder(input_size, mid_size, encoding_dim)
criterion = nn.BCELoss()
optimizer = optim.Adam(autoencoder.parameters(), lr=0.001)

# Train the autoencoder
num_epochs = 200
for epoch in range(num_epochs):
    running_loss = 0.0
    for i, data in enumerate(train_loader, 0):
        inputs, _ = data
        optimizer.zero_grad()
        outputs = autoencoder(inputs)
        loss = criterion(outputs, inputs)
        loss.backward()
        optimizer.step()

        running_loss += loss.item()

    print(f'Epoch {epoch + 1}, Loss: {running_loss / len(train_loader)}')

# Extract the encoder model
class Encoder(nn.Module):
    def __init__(self, encoder):
        super(Encoder, self).__init__()
        self.encoder = encoder

    def forward(self, x):
        x = self.encoder(x)
        return x

encoder_model = Encoder(autoencoder.encoder)

# Encode some data from the test set
encoded_imgs = encoder_model(x_test_tensor).detach().numpy()
